In [1]:
import pandas as pd
import awswrangler as awr
import openpyxl
import os
import shutil
import unicodedata
import win32com.client as win32

#-----------------EXTRACT
path_to_query = r"C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\listagem_mestra_ajustada.sql"

with open (path_to_query,'r') as reading:
    query_readed = reading.read()

df = awr.athena.read_sql_query(query_readed,'silver')

In [2]:
#-----------------TRANSFORM
#adding empty e-mails with the e-mail common pattern
row = df['e_mail'].isnull() & df['unidade'].str.contains('UNIDADE ')
df.loc[row, 'e_mail'] = 'unidade.' + df.loc[row, 'unidade'].str.replace('UNIDADE ', '').str.lower() + '@transdesk.com.br'


In [3]:
#removing empty spaces
df['e_mail'] = df['e_mail'].str.replace(' ','')

#removing diacritics with unicodedata
def remove_diacritics (texto):
    if isinstance(texto,str):
        return ''.join(c for c in unicodedata.normalize('NFKD',texto) if not unicodedata.combining(c))
    return texto

df['e_mail'] = df['e_mail'].apply(remove_diacritics)

df = df.reset_index(drop=True)


In [4]:
df.head()

,cooperativa,unidade,cliente,matricula,conjunto,coverage,beneficio,motorista,id_placa,placa,...,status,status_coverage,benefits_id,categoria,tipo_categoria,faixa_valor,tabela_ativa,tipo_veiculo,tipo,e_mail
0,Stcoop,UNIDADE ITAPEJARA,CRISGUEL TRANSPORTE DE PASSAGEIROS LTDA,12580,6923,42820,ASSISTÊNCIA 24 HORAS,<NA>,3464,MGV6B88,...,ATIVO,ATIVO,27,3 QUARTOS,2X POR ANO - ATÉ 15 ANOS,300Km 2x Ano (150Km ida 150Km volta),S,Micro-Ônibus,Novo,juliano.padilha@transdesk.com.br
1,Stcoop,UNIDADE ITAPEJARA,CRISGUEL TRANSPORTE DE PASSAGEIROS LTDA,12580,6923,42819,REPARAÇÃO OU REPOSIÇÃO DO VEÍCULO,<NA>,3464,MGV6B88,...,ATIVO,ATIVO,24,ÔNIBUS / MICRO-ÔNIBUS,LC: 100% - CP: 4% - TODAS AS MARCAS - TODOS OS...,"De R$100.000,01 até R$125.000,00",S,Micro-Ônibus,Novo,juliano.padilha@transdesk.com.br
2,Stcoop,UNIDADE MARINGÁ,ROGERIO SCHNEIDER,20772,12533,77781,REPARAÇÃO A TERCEIROS,<NA>,8069,MCJ8H68,...,ATIVO,ATIVO,29,PADRÃO,ACIMA DE 7 TONELADAS,DMA 300 MIL/DC 250 MIL/DMO 50 MIL/APP 30 MIL,S,Bitruck,RenovaÃ§Ã£o,unidade.maringa@transdesk.com.br
3,Stcoop,UNIDADE CURITIBA,GRAMAR - GRAMA RONDONIA EIRELI,8297,12535,77812,RASTREADOR REBOQUE/SEMIRREBOQUE,<NA>,<NA>,QTA2F02,...,ATIVO,ATIVO,35,INSTALAÇÃO OBRIGATÓRIA,PADRÃO,TRACKER RF CARRETA,S,Reboque/Semirreboque,RenovaÃ§Ã£o,unidade.curitiba@transdesk.com.br
4,Stcoop,UNIDADE SINOP,LEANDRO MUSSI,21991,8247,51482,ASSESSORIA JURÍDICA,<NA>,24550,RRJ1B83,...,ATIVO,ATIVO,32,PADRÃO,PADRÃO,ASSESSORIA JURÍDICA,S,Cavalo,Novo,unidade.sinop@transdesk.com.br


conjuntos_analise

In [49]:
conjuntos_validos = df.loc[df['beneficio']=='SEGURO DE VIDA', 'conjunto'].unique()


df_filtred = df[df['conjunto'].isin(conjuntos_validos)].copy()
df_filtred2 = df_filtred[df_filtred['benefits_id'].isin([2,6,24,28,40,44])].copy()

df_filtred2['casco'] = df_filtred2['beneficio'].apply(lambda x: x if x != 'SEGURO DE VIDA' else pd.NA)


df_filtred2['seg_vida'] = df_filtred2['beneficio'].apply(lambda x: x if x == "SEGURO DE VIDA" else pd.NA)


df_filtred3 = df_filtred2.groupby(['cooperativa','unidade','matricula','conjunto']).agg(
    qtd_motoristas = ('motorista', 'count'),
    qtd_cascos = ('casco', 'count'),
    qtd_segvida = ('seg_vida', 'count')
).reset_index()

df_filtred3.sort_values(by='conjunto', ascending=False)

df_analysis=df_filtred3[(df_filtred3['qtd_motoristas']!=df_filtred3['qtd_cascos'])|
                      (df_filtred3['qtd_motoristas']!=df_filtred3['qtd_segvida'])|
                      (df_filtred3['qtd_cascos']!=df_filtred3['qtd_segvida'])]

df_analysis.head()
#df_analysis = df_filtred3[row, ['cooperativa','unidade','matricula','conjunto','qtd_motoristas','qtd_cascos','qtd_segvida']]

file_path  = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\conjuntos_analise.xlsx'
df_analysis.to_excel(file_path, engine = 'openpyxl', index = False)


conjuntos_aberto

In [ ]:
df_open = df_filtred2.sort_values(by='conjunto', ascending=False)
file_path  = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\conjuntos_aberto.xlsx'
df_open.to_excel(file_path, engine = 'openpyxl', index = False)

In [45]:
print(
    df_open['motorista'].count(),
    df_open['casco'].count(),
    df_open['seg_vida'].count()
)

10729 10961 10729


In [ ]:
#-----------------LOAD
#Function to send e-mails

def enviar_email(p_email, p_unidade, p_placas, p_motoristas):    
    outlook = win32.Dispatch('outlook.application')  #integrating python with outlook 

    #creating e-mail variable to use
    email = outlook.CreateItem(0)

    #configuring the e-mail - ALTEREI PARA TESTE
    email.To = "dados13@grupounus.com.br; dados03@grupounus.com.br"
    email.Subject = "[CADASTRAMENTO DE MOTORISTAS] - Análise de Dados"
    email.HTMLBody = f"""
    <p>Prezado(a),<p/>

    <p>A unidade {p_unidade}, correspondente ao e-mail {p_email}, possui pendências de cadastramento de motoristas por placa:

    <p>Número de placas cadastradas: {p_placas}
    Número de motoristas cadastrados: {p_motoristas}<p/>

    <p>Favor, cadastrar as placas remanescentes para cômputo na base de dados da empresa, obrigado!<p/>

    <p>Atenciosamente,<p/>

    <p>Equipe Análise de Dados - Grupo Unus<p/>

    <p>(<i>Este é um e-mail automático, por favor não responda<i/>)<p/>
 
    """

    email.Send()

    print("Email enviado")

In [ ]:

df_numeros = df_unidemail_filtered.groupby(['unidade']).agg(
    qtd_placas=('placas', 'sum'),
    qtd_motoristas=('motoristas', 'sum')
).reset_index()

for idx, row in df_unidemail_filtered_test.iterrows():
    iEmail = row['e_mail']
    iUnidade = row['unidade']

    unidade_info = df_numeros.loc[df_numeros['unidade'] == iUnidade].iloc[0]

    iQtdPlacas = unidade_info['qtd_placas']
    iQtdMotoristas = unidade_info['qtd_motoristas']

    # if __name__=="__main__":
    #     enviar_email(iEmail, iUnidade, iQtdPlacas, iQtdMotoristas)

    
    print(f"e_mail:ENVIADO PRA GENTE TESTE/unidade:{iUnidade}/placas:{iQtdPlacas}/motoristas:{iQtdMotoristas}")


        

Email enviado
e_mail:ENVIADO PRA GENTE TESTE/unidade:TS CONSULTORIA EM TRANSPORTES LTDA/placas:939/motoristas:218
Email enviado
e_mail:ENVIADO PRA GENTE TESTE/unidade:UNIDADE ARACAJU/placas:14855/motoristas:1558
Email enviado
e_mail:ENVIADO PRA GENTE TESTE/unidade:UNIDADE ARACAJU/placas:14855/motoristas:1558
Email enviado
e_mail:ENVIADO PRA GENTE TESTE/unidade:UNIDADE ARACAJU/placas:14855/motoristas:1558
Email enviado
e_mail:ENVIADO PRA GENTE TESTE/unidade:UNIDADE CAMPO GRANDE/placas:2123/motoristas:110


In [ ]:


path_to_save = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista'
file_name = 'Cobertura de Seguros Motoristas.xlsx'
final_path = os.path.join(path_to_save, file_name)
os.remove(final_path)
df.to_excel(final_path, engine='openpyxl', index = False)